In [24]:
import pandas as pd
import requests
from functools import cache

In [25]:
URL = 'https://api.mapy.cz/v1/rgeocode'

API_KLIC = #klíč lze získat po registrování na mapy.cz

LANG = 'cs'


In [26]:
#uložení stažených dat jako json, použití cache pro zlepšení výkonu
@cache
def rgeo(lon: float, lat: float) -> tuple[str, str]: 
    r = requests.get(URL, params={
        'lang': LANG,
    'lon': lon,
    'lat': lat,
    'apikey': API_KLIC,
    })
    # print(r.status_code, r.reason)
    data = r.json()

    return data

In [27]:
#otevření souboru, který chceme geokodovat

df = pd.read_csv("novy_grid.csv", encoding="utf-8")

In [28]:
df.head()

,distance,latitude,longitude,geometry,id_cukrarna,distance_mhd,name_mhd
0,2.400607,49.9468,14.3894,POINT (14.3894 49.9468),271,0.567415,"Jíloviště,Výzkumný ústav"
1,2.548011,49.9468,14.3944,POINT (14.3944 49.9468),271,0.128849,Strnady
2,4.010847,49.9518,14.3394,POINT (14.3394 49.9518),589,0.179248,Kazín
3,3.769092,49.9518,14.3444,POINT (14.3444 49.9518),589,0.405534,Kazín
4,3.597397,49.9518,14.3494,POINT (14.3494 49.9518),589,0.936626,Ke Kazínu


In [29]:
#zkouška pro jeden bod
rgeo(14.3894, 49.9468)

{'items': [{'name': 'Praha-Zbraslav',
   'label': 'Městská část',
   'position': {'lon': 14.3799, 'lat': 49.97403},
   'type': 'regional.municipality_part',
   'location': 'Praha, Česko',
   'regionalStructure': [{'name': 'Praha-Zbraslav',
     'type': 'regional.municipality_part'},
    {'name': 'Praha', 'type': 'regional.municipality'},
    {'name': 'okres Hlavní město Praha', 'type': 'regional.region'},
    {'name': 'kraj Hlavní město Praha', 'type': 'regional.region'},
    {'name': 'Česko', 'type': 'regional.country', 'isoCode': 'CZ'}]}],
 'location_boxes': {}}

In [30]:
#vybrání dat která nás zajímají (městská část, ulice a číslo popisné)
def rgeo_to_name_and_zip(data):
    name, street, number = None, None, None
    for i in data['items'][0]['regionalStructure']:
        if i['type'] == 'regional.municipality_part':
            name = i['name'].replace('Praha-', '')
        if i['type'] == 'regional.street':
            street = i['name']
        if i['type'] == 'regional.address':
            number = i['name']
    # if 'zip' in data['items'][0]:        
    #     _zip = data['items'][0]['zip']
    return name, street, number

In [31]:
#zkouška na části datasetu
#df = df[:100]

In [32]:
# geokodování našich dat
df["rgeo_data"] = df.apply(lambda row: rgeo(row.longitude, row.latitude), axis=1)
df[["name", "street", "number"]] = df["rgeo_data"].apply(rgeo_to_name_and_zip).apply(pd.Series)
df.drop(columns=["rgeo_data"], inplace=True)

In [33]:
#kontrola dat
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   distance      100 non-null    float64
 1   latitude      100 non-null    float64
 2   longitude     100 non-null    float64
 3   geometry      100 non-null    object 
 4   id_cukrarna   100 non-null    int64  
 5   distance_mhd  100 non-null    float64
 6   name_mhd      100 non-null    object 
 7   name          92 non-null     object 
 8   street        59 non-null     object 
 9   number        54 non-null     object 
dtypes: float64(4), int64(1), object(5)
memory usage: 7.9+ KB


In [34]:
# uložení do souboru csv
df.to_csv("soubory/kompletni_data.csv")